In [22]:
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *

In [3]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [67]:
import numpy as np
# get detailed data from individual movie webpage

def get_movie_brief(soup):
    
    headers=['movie_title', 'rating', 'vote', 'certificate', 'genre', 'release_date', 'metascore', 'keywords','budget',\
            'opening_weekend_usa','gross_usa','gross_world']
    
    # find movie title
    title = np.nan
    try:
        title = " ".join(soup.find('h1').text.split()[:-1]) ## problem?
    except:
        pass
    
    # find rating
    rating = np.nan
    try:
        rating = float(soup.find('span',attrs={'itemprop':'ratingValue'}).text)
    except:
        pass
    
    # find vote (rating count)
    vote = np.nan
    try:
        vote = int(soup.find('span',attrs={'itemprop':'ratingCount'}).text.replace(',',''))
    except:
        pass
        
    # find content rating
    certificate = np.nan
    try:
        certificate = soup.find('div', class_="subtext").text.split()[0]
    except:
        pass
        
    # find list of genre
    genre_list=[]
    try:
        for genres in soup.find('div', class_="subtext").find_all('a')[:-1]:
            genre_list.append(genres.text)
    except:
        pass
            
    # find release date
    date = np.nan
    try:
        date_pre = soup.find('div', class_="subtext").find_all('a')[-1].text.split('(')[0]
        date = pd.to_datetime(date_pre) ## why is it Timestamp? format ='%d-%B-%Y'
    except:
        pass
    
    # find metascorre
    if soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem") is not None:
        meta = int(soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem").text.strip('\n'))
    else:
        meta = np.nan
        
        
    # find plot keywords
    keyword_list=[]
    try:
        for keywords in soup.find('div', class_="article", id="titleStoryLine").\
        find('div', class_="see-more inline canwrap").find_all('a')[:-1]:
            keyword_list.append(keywords.text.strip(' '))
    except:
        pass
        
    
    
    # find budget, opening weekend USA, gross USA, cumulative worldwide gross
    # assign default value:
    budget, opening, gross_usa, gross_cw, distributor = np.nan, np.nan, np.nan, np.nan, np.nan
    try:
        for line in soup.find('div', class_="article", id="titleDetails").find_all('h4'):        
            if "Budget:" in line:
                budget = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Opening Weekend USA:" in line:
                opening = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Gross USA:" in line:
                gross_usa = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Cumulative Worldwide Gross:" in line:
                gross_cw = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Production Co:" in line:
                distributor = line.findNext().text.replace(' ','')
    except:
        pass

#     # find runtime
#     try:
#         if soup.find('div', class_="title_wrapper").find('time') is not None:
#             rt = soup.find('div', class_="title_wrapper").find('time').text.replace('\n','').split()
#             hr = "".join(filter(str.isdigit, rt[0]))
#             if len(rt) > 2:
#                 min = "".join(filter(str.isdigit, rt[1]))
#             else:
#                 min = "0"
#             runtime = int(hr)*60 + int(min)
#         else:
#             runtime = np.nan
#     except:
#         pass
        
        
        
        
#     # find director
#     director= np.nan
#     try:
#         director = soup.find('div',class_="credit_summary_item").find('a').text
#         link_d = soup.find('div',class_="credit_summary_item").find('a').get('href')
#     except:
#         pass
    
#     # find writer
#     writer = np.nan
#     writer_line = soup.find_all('div',class_="credit_summary_item")[1].find_all('a')
#     link_w = [w.get('href') for w in writer_line]
#     writer = [w.text for w in writer_line]
#     if '1 more credit' in writer:
#         writer.remove('1 more credit')
#         link_w.pop()
        
#     # find star
#     star = np.nan
#     star_line = soup.find_all('div',class_="credit_summary_item")[2].find_all('a')
#     link_s = [s.get('href') for s in star_line]
#     star = [s.text for s in star_line]
#     if 'See full cast & crew' in star:
#         star.remove('See full cast & crew')
#         link_s.pop()
    
    
    
#     # find language
#     language= np.nan
#     t= []
#     matching = []
#     for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
#         t.append(div.text.replace('\n','').replace(' ',''))
    
#     matching = [s for s in t if 'Language:' in s]
#     language = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
    
#     # find country
#     country= np.nan
#     t= []
#     matching = []
#     for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
#         t.append(div.text.replace('\n','').replace(' ',''))
    
#     matching = [s for s in t if 'Country:' in s]
#     country = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
        
    movie_dict = dict(zip(headers, [title,
                                    rating,
                                    vote,
                                   certificate,
                                   genre_list,
                                   date,
                                   meta,
                                   keyword_list,
                                   budget,
                                   opening,
                                   gross_usa,
                                   gross_cw]))
    
    return movie_dict

In [4]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [5]:
# # Open automated Chrome with fictionDB webpage
# Test with The Godfather
movie_url = "https://www.imdb.com/title/tt0068646/"
driver = webdriver.Chrome(chromedriver)
driver.get(movie_url)

In [8]:
driver.find_element_by_xpath('//a[@href="'+url+'"]')

In [ ]:
<a href="/name/nm0000338/?ref_=tt_ov_dr">Francis Ford Coppola</a>

In [12]:
"//div[@class='credit_summary_item']//h4/following-sibling::a"

"//div[@class='credit_summary_item']//h4/following-sibling::a"

In [18]:
# director = driver.find_element_by_xpath("//*[text()='Director:']")
director = driver.find_element_by_xpath("//div[@class='credit_summary_item']//h4/following-sibling::a")
director.click()

In [19]:
l=driver.find_element_by_xpath("//*[text()='Director:']//h1/following-sibling::h2")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[text()='Director:']//h1/following-sibling::h2"}
  (Session info: chrome=87.0.4280.141)


In [9]:
search('Jane Austen')

{'Jane Austen': [1818,
  1817,
  1815,
  1814,
  1813,
  1811,
  1805,
  1805,
  1803,
  1790,
  1786]}

In [3]:
di={}
di["jane"] = [1,2,3]

In [4]:
di

{'jane': [1, 2, 3]}

In [3]:
try:
    publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
    dates = [s.text.split('-') for s in publication]
    years = [int(y[-1]) for y in dates]
except:
    print("can't find publication")
    pass

In [4]:
years

[1818, 1817, 1815, 1814, 1813, 1811, 1805, 1805, 1803, 1790, 1786]

In [6]:
di={}
author="Jane"
di[author] = years

In [7]:
di

{'Jane': [1818, 1817, 1815, 1814, 1813, 1811, 1805, 1805, 1803, 1790, 1786]}

In [23]:
director_url="https://www.imdb.com/name/nm0000338/?ref_=tt_ov_dr#director"
director_soup = get_soup(director_url)

In [24]:
director_soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///name/nm0000338?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Francis Ford Coppola - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.

In [29]:
films = director_soup.find('div',id="filmo-head-director").findNext('div')

In [72]:
# get detailed data from individual movie webpage

def get_movie_detail(soup):
    
    headers=['movie_title', 'rating', 'vote', 'certificate', 'genre', 'release_date', 'metascore', 'keywords','budget',\
            'opening_weekend_usa','gross_usa','gross_world','runtime','director','link_d','writer','link_w',\
             'star','link_s','distributor','language','country']
    
    # find movie title
    title = " ".join(soup.find('h1').text.split()[:-1]) ## problem?
    
    # find rating
    rating = float(soup.find('span',attrs={'itemprop':'ratingValue'}).text)
    
    # find vote (rating count)
    vote = int(soup.find('span',attrs={'itemprop':'ratingCount'}).text.replace(',',''))
    
    # find content rating
    certificate = soup.find('div', class_="subtext").text.split()[0]
    
    # find list of genre
    genre_list=[]
    for genres in soup.find('div', class_="subtext").find_all('a')[:-1]:
        genre_list.append(genres.text)
        
    # find release date
    date_pre = soup.find('div', class_="subtext").find_all('a')[-1].text.split('(')[0]
    date = pd.to_datetime(date_pre) ## why is it Timestamp? format ='%d-%B-%Y'
    
    # find metascorre
    if soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem") is not None:
        meta = int(soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem").text.strip('\n'))
    else:
        meta = np.nan
        
        
    # find plot keywords
    keyword_list=[]
    for keywords in soup.find('div', class_="article", id="titleStoryLine").\
    find('div', class_="see-more inline canwrap").find_all('a')[:-1]:
        keyword_list.append(keywords.text.strip(' '))
        
    
    
    # find budget, opening weekend USA, gross USA, cumulative worldwide gross
    # assign default value:
    budget, opening, gross_usa, gross_cw, distributor = np.nan, np.nan, np.nan, np.nan, np.nan
    for line in soup.find('div', class_="article", id="titleDetails").find_all('h4'):        
        if "Budget:" in line:
            budget = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Opening Weekend USA:" in line:
            opening = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Gross USA:" in line:
            gross_usa = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Cumulative Worldwide Gross:" in line:
            gross_cw = int(''.join(s for s in line.next_sibling if s.isdigit()))
        if "Production Co:" in line:
            distributor = line.findNext().text.replace(' ','')

    # find runtime
    if soup.find('div', class_="title_wrapper").find('time') is not None:
        rt = soup.find('div', class_="title_wrapper").find('time').text.replace('\n','').split()
        hr = "".join(filter(str.isdigit, rt[0]))
        if len(rt) > 2:
            min = "".join(filter(str.isdigit, rt[1]))
        else:
            min = "0"
        runtime = int(hr)*60 + int(min)
    else:
        runtime = np.nan
        
        
        
        
    # find director
    director= np.nan
    director = soup.find('div',class_="credit_summary_item").find('a').text
    link_d = soup.find('div',class_="credit_summary_item").find('a').get('href')
    
    # find writer
    writer = np.nan
    writer_line = soup.find_all('div',class_="credit_summary_item")[1].find_all('a')
    link_w = [w.get('href') for w in writer_line]
    writer = [w.text for w in writer_line]
    if '1 more credit' in writer:
        writer.remove('1 more credit')
        link_w.pop()
        
    # find star
    star = np.nan
    star_line = soup.find_all('div',class_="credit_summary_item")[2].find_all('a')
    link_s = [s.get('href') for s in star_line]
    star = [s.text for s in star_line]
    if 'See full cast & crew' in star:
        star.remove('See full cast & crew')
        link_s.pop()
    
    
    
    # find language
    language= np.nan
    t= []
    matching = []
    for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
        t.append(div.text.replace('\n','').replace(' ',''))
    
    matching = [s for s in t if 'Language:' in s]
    language = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
    
    # find country
    country= np.nan
    t= []
    matching = []
    for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
        t.append(div.text.replace('\n','').replace(' ',''))
    
    matching = [s for s in t if 'Country:' in s]
    country = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
        
    movie_dict = dict(zip(headers, [title,
                                    rating,
                                    vote,
                                   certificate,
                                   genre_list,
                                   date,
                                   meta,
                                   keyword_list,
                                   budget,
                                   opening,
                                   gross_usa,
                                   gross_cw,
                                   runtime,
                                   director,
                                    link_d,
                                    writer,
                                    link_w,
                                    star,
                                    link_s,
                                   distributor,
                                   language,
                                   country]))
    
    return movie_dict

In [92]:
base_url = "https://www.imdb.com"
director = "Francis Ford Coppola"
director_url="https://www.imdb.com/name/nm0000338/?ref_=tt_ov_dr#director"
director_soup = get_soup(director_url)
films = director_soup.find('div',id="filmo-head-director").findNext('div')
director_dict={}
movies=[]
headers = ['director','movie','year',]
for filmo in films:
    movie = filmo.find('a')
    if type(movie) != int:
        movie_title.append(movie.text)
        movie_link = (movie.get('href'))
        
        soup = get_soup(base_url + movie_link)
        movie_detail = get_movie_brief(soup)
        movies.append(movie_detail)

director_dict[director] = movies

In [93]:
director_dict

{'Francis Ford Coppola': [{'movie_title': '',
   'rating': nan,
   'vote': nan,
   'certificate': 'Drama,',
   'genre': ['Drama'],
   'release_date': nan,
   'metascore': nan,
   'keywords': ['utopia',
    'architect',
    'architect as protagonist',
    "architect's plan",
    'new york city'],
   'budget': nan,
   'opening_weekend_usa': nan,
   'gross_usa': nan,
   'gross_world': nan},
  {'movie_title': 'Distant Vision',
   'rating': 6.2,
   'vote': 105,
   'certificate': 'Drama',
   'genre': ['Drama'],
   'release_date': Timestamp('2016-07-22 00:00:00'),
   'metascore': nan,
   'keywords': ['teenager'],
   'budget': nan,
   'opening_weekend_usa': nan,
   'gross_usa': nan,
   'gross_world': nan},
  {'movie_title': 'Twixt',
   'rating': 4.8,
   'vote': 10415,
   'certificate': 'R',
   'genre': ['Comedy', 'Fantasy', 'Horror'],
   'release_date': Timestamp('2012-04-11 00:00:00'),
   'metascore': nan,
   'keywords': ['ghost', 'writer', 'murder', 'stake', 'hotel'],
   'budget': 7000000,
 

In [94]:
import pandas as pd
ddd = pd.DataFrame(director_dict)

In [95]:
ddd

,Francis Ford Coppola
0,"{'movie_title': '', 'rating': nan, 'vote': nan..."
1,"{'movie_title': 'Distant Vision', 'rating': 6...."
2,"{'movie_title': 'Twixt', 'rating': 4.8, 'vote'..."
3,"{'movie_title': 'Tetro', 'rating': 6.8, 'vote'..."
4,"{'movie_title': 'Youth Without Youth', 'rating..."
5,{'movie_title': 'Un matin partout dans le mond...
6,"{'movie_title': 'The Rainmaker', 'rating': 7.2..."
7,"{'movie_title': 'Jack', 'rating': 5.8, 'vote':..."
8,{'movie_title': 'Making 'Bram Stoker's Dracula...
9,"{'movie_title': 'Bram Stoker's Dracula', 'rati..."


'https://www.imdb.com/title/title/tt10128846/'

'/'

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>404 Error - IMDb</title>
<style>
html, body {
  height: 100%;
}
body {
  margin: auto;
  width: 1008px;
  background-color: #D4D9DD;
  font-family: Verdana, Arial, sans-serif;
}
a {
  text-decoration: none;
}
a:hover {
  text-decoration: underline;
}
a:link, a:hover, a:visited {
  color: #136CB2;
}
a:active {
  color: #E7BE00;
}
#error {
  height: 100%;
  background-color: white;
  border-left: 1px solid #999999;
  border-right: 1px solid #999999;
  box-shadow: 0 0 5px 5px #C5CACD;
}
.error_message {
  color: #999999;
  font-size: 17.5px;
  padding: 30px 70px 20px;
}
.error_bubble {
  font-family: Arial, helvetica, sans-serif;
  font-weight: bold;
  border-radius: 8px;
  -moz-border-radius: 8px;
  margin: 0 70px 0 70px;
  padding: 50px;
}
.error_bubble div {
  display: inline-block;
  vertical-align: middle;
}
.error_quote {
  color: #FFFFFF;
  font-size: 30px;
  line-height: 1.35em;
  padding-left: 50px;
  width: 5

{'director': 'Francis Ford Coppola',
 'movies': [{'movie_title': nan,
   'rating': nan,
   'vote': nan,
   'certificate': nan,
   'genre': [],
   'release_date': nan,
   'metascore': nan,
   'keywords': [],
   'budget': nan,
   'opening_weekend_usa': nan,
   'gross_usa': nan,
   'gross_world': nan},
  {'movie_title': nan,
   'rating': nan,
   'vote': nan,
   'certificate': nan,
   'genre': [],
   'release_date': nan,
   'metascore': nan,
   'keywords': [],
   'budget': nan,
   'opening_weekend_usa': nan,
   'gross_usa': nan,
   'gross_world': nan},
  {'movie_title': nan,
   'rating': nan,
   'vote': nan,
   'certificate': nan,
   'genre': [],
   'release_date': nan,
   'metascore': nan,
   'keywords': [],
   'budget': nan,
   'opening_weekend_usa': nan,
   'gross_usa': nan,
   'gross_world': nan},
  {'movie_title': nan,
   'rating': nan,
   'vote': nan,
   'certificate': nan,
   'genre': [],
   'release_date': nan,
   'metascore': nan,
   'keywords': [],
   'budget': nan,
   'opening_w

{'movie': ['Megalopolis',
  'Distant Vision',
  'Twixt',
  'Tetro',
  'Youth Without Youth',
  'Un matin partout dans le monde',
  'The Rainmaker',
  'Jack',
  "Making 'Bram Stoker's Dracula'",
  "Bram Stoker's Dracula",
  'The Godfather Trilogy: 1901-1980',
  'The Godfather: Part III',
  'New York Stories',
  'Tucker: The Man and His Dream',
  'Gardens of Stone',
  'Faerie Tale Theatre',
  'Peggy Sue Got Married',
  'Captain EO',
  'The Cotton Club',
  'Rumble Fish',
  'The Outsiders',
  'One from the Heart',
  'Apocalypse Now',
  'The Godfather: A Novel for Television',
  'The Godfather: Part II',
  'The Conversation',
  'The Godfather',
  'The Rain People',
  "Finian's Rainbow",
  "You're a Big Boy Now",
  'Dementia 13',
  'The Terror',
  'Tonight for Sure',
  'The Bellboy and the Playgirls',
  'Battle Beyond the Sun',
  'No Cigar'],
 'link': ['/title/tt10128846/',
  '/title/tt2937692/',
  '/title/tt1756851/',
  '/title/tt0964185/',
  '/title/tt0481797/',
  '/title/tt3225442/',
  '/